In [2]:
!nvidia-smi

Mon Aug 11 13:00:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from itertools import combinations
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import ast
import random
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
train_df = pd.read_csv("/content/drive/MyDrive/menu_rec/orders_grouped_cleaned(1).csv")

In [11]:
# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [13]:
train_df['parsed_items'] = train_df['item_name'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
order_df = train_df.groupby(['ORDER_ID', 'CUSTOMER_ID'])['parsed_items'].sum().reset_index()

# Uniqueness in cart items
order_df['parsed_items'] = order_df['parsed_items'].apply(lambda x: list(set(x)))

# Filtering out carts with less than 4 items
order_df = order_df[order_df['parsed_items'].str.len() >= 4]

# Separating the target and the context
order_df['target_item'] = order_df['parsed_items'].apply(lambda x: x[3])
order_df['context_items'] = order_df['parsed_items'].apply(lambda x: x[:3])

def dropout_cart(cart, p=0.25):
    if len(cart) <= 2:
        return cart
    return [item for item in cart if random.random() > p]

order_df['context_items'] = order_df['context_items'].apply(lambda x: dropout_cart(x))

# Final sequences for training
item_sequences = order_df['context_items'].tolist()
target_items = order_df['target_item'].tolist()

In [14]:
#Word2Vec Model
w2v_model = Word2Vec(
    sentences=item_sequences,
    vector_size=100,
    window=3,
    min_count=1,
    workers=4,
    sg=1,
    negative=5,
    sample=1e-3,       # Downsampling frequent words
    alpha=0.03,
    min_alpha=0.0007,
    epochs=20
)

In [17]:
# Fine-tune with test feedback
test_df_feedback = pd.read_excel("/content/drive/MyDrive/menu_rec/test_data_question.csv (1).xlsx")

In [18]:
test_df_feedback['cart_items'] = test_df_feedback[['item1', 'item2', 'item3']].values.tolist()
test_df_feedback['actual_missing_item'] = test_df_feedback['item4']

fine_tune_data = [row['cart_items'] + [row['actual_missing_item']] for _, row in test_df_feedback.iterrows()]
if fine_tune_data:
    w2v_model.build_vocab(fine_tune_data, update=True)
    w2v_model.train(fine_tune_data, total_examples=len(fine_tune_data), epochs=3)


In [19]:
#Feature Engineering + Co-occurrence
def get_cart_vector(cart):
    vectors = [w2v_model.wv[i] for i in cart if i in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

def cosine_similarity(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

item_counts = Counter([item for cart in item_sequences for item in cart])
item_popularity = pd.Series(item_counts)

cooccur = defaultdict(Counter)
for cart in item_sequences:
    for i, j in combinations(set(cart), 2):
        cooccur[i][j] += 1
        cooccur[j][i] += 1

def get_item_category(item):
    item = item.lower()
    if "fries" in item or "strips" in item or "sticks" in item:
        return "side"
    elif "drink" in item or "soda" in item or "beverage" in item or "20 oz" in item:
        return "drink"
    elif "combo" in item or "meal" in item or "feast" in item or "Pack" in item:
        return "combo"
    elif "wings" in item or "sub" in item or "crispy" in item:
        return "main"
    else:
        return "other"

def get_cart_category_vector(cart_items):
    return Counter([get_item_category(i) for i in cart_items])

In [20]:
# Build ranking dataset
rerank_data = []
for _, row in test_df_feedback.iterrows():
    cart_items = row['cart_items']
    actual = row['actual_missing_item']
    cart_len = len(cart_items)
    cart_cat = get_cart_category_vector(cart_items)

    frequent_items = item_popularity[item_popularity > 5].index.tolist()
    negative_candidates = list(set(frequent_items) - set(cart_items) - {actual})
    hard_negatives = random.sample(negative_candidates, min(40, len(negative_candidates)))

    for label, item in [(1, actual)] + [(0, neg) for neg in hard_negatives]:
        if item not in w2v_model.wv:
            continue

        sims = [cosine_similarity(w2v_model.wv[item], w2v_model.wv[ci]) for ci in cart_items if ci in w2v_model.wv]
        if not sims:
            continue

        rerank_data.append({
            'ORDER_ID': row['ORDER_ID'],
            'sim_avg': np.mean(sims),
            'sim_max': np.max(sims),
            'sim_min': np.min(sims),
            'sim_std': np.std(sims),
            'sim_len': len(sims),
            'popularity': np.log1p(item_popularity.get(item, 0)),
            'cooccur_sum': sum(cooccur[item][ci] for ci in cart_items),
            'mean_cooccur': (sum(cooccur[item][ci] for ci in cart_items) / cart_len) if cart_len else 0,
            'cart_len': cart_len,
            'cat_mismatch': int(get_item_category(item) not in cart_cat),
            'hit': label
        })

rerank_df = pd.DataFrame(rerank_data)


In [21]:
# Train LambdaRank model
features = [col for col in rerank_df.columns if col not in ['hit', 'ORDER_ID']]
X = rerank_df[features]
y = rerank_df['hit']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
group_train = [41] * (len(X_train) // 41)

model = lgb.LGBMRanker(
    objective="lambdarank",
    n_estimators=80,
    learning_rate=0.05,
    importance_type='gain',
    random_state=SEED
)

model.fit(X_train[:len(group_train)*41], y_train[:len(group_train)*41], group=group_train)
print("LambdaRank model trained")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000878 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1636
[LightGBM] [Info] Number of data points in the train set: 31980, number of used features: 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped trainin

In [25]:
#Re-rank and evaluate
def rerank_candidates(cart_items, top_k=3):
    cart_vec = get_cart_vector(cart_items)
    cart_cat = get_cart_category_vector(cart_items)
    cart_len = len(cart_items)

    # Candidates with duplicates
    candidate_list = []
    for ci in cart_items:
        candidate_list.extend(cooccur[ci].keys())  # keep all occurrences
    candidates = list(dict.fromkeys(candidate_list))  # preserve order and remove only exact dupes at end
    candidates = [c for c in candidates if c not in cart_items]

    candidate_scores = []
    for item in candidates:
        if item not in w2v_model.wv:
            continue

        sims = [cosine_similarity(w2v_model.wv[item], w2v_model.wv[ci])
                for ci in cart_items if ci in w2v_model.wv]
        if not sims:
            continue

        sim_avg = np.mean(sims)
        sim_max = np.max(sims)
        sim_min = np.min(sims)
        sim_std = np.std(sims)
        sim_len = len(sims)
        pop = np.log1p(item_popularity.get(item, 0))
        cooccur_sum = sum([cooccur[item][ci] for ci in cart_items])
        mean_cooccur = cooccur_sum / cart_len if cart_len else 0
        item_cat = get_item_category(item)
        cat_mismatch = int(item_cat not in cart_cat)


        features_arr = np.array([[sim_avg, sim_max, sim_min, sim_std, sim_len,
                                  pop, cooccur_sum, mean_cooccur, cart_len, cat_mismatch]])
        prob = model.predict(features_arr)[0]
        candidate_scores.append((item, prob))

   # Top K predictions
    top_preds = sorted(candidate_scores, key=lambda x: x[1], reverse=True)[:top_k]
    pred_items = [item for item, _ in top_preds]

    # Backup in case of shortage
    if len(pred_items) < top_k:
        backup = [i for i in item_popularity.index if i not in cart_items]
        pred_items += backup[:top_k - len(pred_items)]

    return pred_items[:top_k]

In [26]:
# Evaluate Recall@3
correct = 0
total = len(test_df_feedback)

for _, row in test_df_feedback.iterrows():
    cart = row['cart_items']
    actual = row['actual_missing_item']
    preds = rerank_candidates(cart, top_k=3)  # explicitly set top_k
    if actual in preds:
        correct += 1

recall_at_3 = correct / total * 100
print(f"Final Recall@3 (LambdaRank): {recall_at_3:.2f}%")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/pyth

Final Recall@3 (LambdaRank): 0.00%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRanker was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:27

In [27]:
print(f"Final Recall@3 (LambdaRank): {recall_at_3:.2f}%")

Final Recall@3 (LambdaRank): 0.00%
